In [2]:
import os
import pandas as pd
import subprocess
import re
import time
from PIL import Image
import matplotlib.pyplot as plt

In [3]:
def identify():
    while True:
        protein_in = "_".join(input("please input your protein name\n\t").split())
        taxonomy_in = "_".join(input("please input the taxonomic group you want to analyse\n\t").split())
        if protein_in == "" or taxonomy_in == "":
            print("\n\nYou missed at least one input, I CAN'T create something out of nothing ,sorry! Please try again!!!\n\n")
            continue
        else:
            break
    return protein_in, taxonomy_in

In [4]:
# prepare a check function to see if the user want to continue or not, if user didn't input something right, notify user
def check_continue(information = "do you want to continue? \n\t"):
    while True:
        user_info = input(information + " [yes/no]\n\t")
        if user_info.upper().startswith("Y"):
            return True
        elif user_info.upper().startswith("N"):
            return False
        else: 
            print("\n\tyou didn't input correctly, please input 'yes' or 'no'! \t")

In [5]:
def download_file(protein_ask = "pyruvate_dehydrogenase", taxonomy_ask = "ascomycete_fungi"):
    while True:
        result_readable = subprocess.check_output(f"esearch -db protein -query '{protein_ask}[Protein Name] AND {taxonomy_ask}[Organism]'", shell = True).decode("utf-8")
        search_num = re.search(r"<Count>(.*?)</Count>", result_readable).group(1)
        if int(search_num) >= 1000:
            print("\nSorry but there are too many sequences found based on your search criteria, which is hard to process, could you please try again?\n")
            protein_ask, taxonomy_ask = identify()
            continue
        elif check_continue(f"you have found {search_num} sequences, do you wanna continue?"):
            print("downloading...... in a second~")
            break
        else:
            print("Since you don't want to continue, you are going to exit in 3 seconds. See you next time~")
            time.sleep(3)
            exit()
    os.system(f"esearch -db protein -query '{protein_ask}[Protein Name] AND {taxonomy_ask}[Organism]' | efetch -format fasta > {protein_ask}_{taxonomy_ask}.fa")
    return print(f"\n###{protein_ask}_{taxonomy_ask}.fa### has been successfully downloaded to your current working directory")


In [6]:
def pullseq(protein_name, taxonomy_name, n = 150):
    os.system(f"./pullseq -i {protein_name}_{taxonomy_name}_ori.fa -m {n}")

In [7]:
protein, taxonomy = identify()

download_file(protein, taxonomy)

please input your protein name
	pyruvate dehydrogenase
please input the taxonomic group you want to analyse
	ascomycete fungi


curl: (6) Could not resolve host: eutils.ncbi.nlm.nih.gov
 ERROR:  curl command failed ( Sat 19 Nov 14:07:27 GMT 2022 ) with: 6
-X POST https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi -d retmax=0&usehistory=y&db=protein&term=pyruvate_dehydrogenase%5BProtein%20Name%5D%20AND%20ascomycete_fungi%20%5BOrganism%5D&tool=edirect&edirect=18.1&edirect_os=Linux&email=s2328491%40bioinfmsc8.bio.ed.ac.uk
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esearch.fcgi -retmax 0 -usehistory y -db protein -term "pyruvate_dehydrogenase[Protein Name] AND ascomycete_fungi [Organism]" -tool edirect -edirect 18.1 -edirect_os Linux -email s2328491@bioinfmsc8.bio.ed.ac.uk
EMPTY RESULT
SECOND ATTEMPT


you have found 232 sequences, do you wanna continue? [yes/no]
	yes
downloading...... in a second~

###pyruvate_dehydrogenase_ascomycete_fungi.fa### has been successfully downloaded to your current working directory


In [18]:
file_fullname = f"{protein}_{taxonomy}_ori.fa"

In [19]:
file_fullname

'pyruvate_dehydrogenase_ascomycete_fungi_ori.fa'

In [8]:
print(protein, taxonomy)

pyruvate_dehydrogenase ascomycete_fungi


In [8]:
with open(f"{protein}_{taxonomy}.fa") as my_file:
    species = []
    acc_nums = []
    seqs = []
    current_seq = ""
    for file_line in my_file:
        if file_line.startswith(">"):
            if current_seq != "":
                seqs.append(current_seq)
            current_seq = ""
            acc_num = re.search(r">([^\s]*)", file_line)
            if acc_num == None:
                acc_num = "-"
            else:
                acc_num = acc_num.group(1)
            organism = re.search(r"\[(.*)]", file_line)
            if organism == None:
                organism = "-"
            else:
                organism = organism.group(1)
            # 如果没有搜索到相应的内容，进行置空处理
            acc_nums.append(acc_num)
            species.append(organism)
#         elif current_seq == "":
#             current_seq = file_line.strip()
        else:
            current_seq += file_line.strip()
    seqs.append(current_seq)

In [9]:
seqlen = []
i = 0
for seq in seqs:
    seqlen.append(len(seq))
species_unrep_num = len(set(species))
species_unrep = list(set(species))
if check_continue(f"\n...\n...\nThere are {species_unrep_num} unduplicated species in the file you just downloaded, which are stored in a list, do you want to check them or just quit?"):
    for spe in sorted(species_unrep):
        print(spe)
else:    
    print("Since you don't want to continue, you are going to exit in 3 seconds. See you next time~")
    time.sleep(3)
    exit()


...
...
There are 130 unduplicated species in the file you just downloaded, which are stored in a list, do you want to check them or just quit? [yes/no]
	yes
Ambrosiozyma platypodis
Ascobolus immersus RN42
Ascodesmis nigricans
Ascoidea rubescens DSM 1968
Aspergillus aculeatinus CBS 121060
Aspergillus arachidicola
Aspergillus bombycis
Aspergillus clavatus NRRL 1
Aspergillus costaricaensis CBS 115574
Aspergillus eucalypticola CBS 122712
Aspergillus fischeri NRRL 181
Aspergillus flavus
Aspergillus fumigatus Z5
Aspergillus heteromorphus CBS 117.55
Aspergillus luchuensis
Aspergillus luchuensis IFO 4308
Aspergillus neoniger CBS 115656
Aspergillus niger
Aspergillus niger ATCC 13496
Aspergillus niger CBS 101883
Aspergillus niger CBS 513.88
Aspergillus nomiae NRRL 13137
Aspergillus novofumigatus IBT 16806
Aspergillus phoenicis ATCC 13157
Aspergillus piperis CBS 112811
Aspergillus sclerotialis
Aspergillus sclerotiicarbonarius CBS 121057
Aspergillus sp. HF37
Aspergillus steynii IBT 23096
Aspergi

In [10]:
all_data = {"Accession Number": acc_nums, "organisms" : species, "length of sequences" : seqlen, "Sequences" : seqs}
df_all_data = pd.DataFrame(all_data)
if check_continue("\n...\n...\n...\nAll the data has been organized in a form, do you wanna generate a tsv file?"):
    df_all_data.to_csv("all_data_you_searched.tsv", sep = '\t', header = True)
    print("\ntsv file named 'all_data_you_searched.tsv' has been generated in the current working dictionary")


...
...
...
All the data has been organized in a form, do you wanna generate a tsv file? [yes/no]
	yes

tsv file named 'all_data_you_searched.tsv' has been generated in the current working dictionary


In [13]:
df_all_data.sort_values('organisms', ascending = True, inplace = True)
df_all_data

,Accession Number,organisms,length of sequences,Sequences
147,AGO87542.1,Ambrosiozyma platypodis,132,FYGGNGIVGAQVPLGAGLAFAHHYRNNSNCTFTLYGDGAANQGQVF...
93,RPA75706.1,Ascobolus immersus RN42,500,MTVASLFTRAAARSATKSTVGKSILGSTVVTPSTIARPQLYASLLQ...
89,TGZ84763.1,Ascodesmis nigricans,426,MPALSPTMTAGNIGTWQKQPGDAINPGDVLVEIETDKAQMDFEFQE...
182,ODV59076.1,Ascoidea rubescens DSM 1968,512,MSAIALKSKSLNPLLKCSLPILKNSHSSQLFLKNSYNYLLLNKSFS...
171,XP_020045383.1,Ascoidea rubescens DSM 1968,512,MSAIALKSKSLNPLLKCSLPILKNSHSSQLFLKNSYNYLLLNKSFS...
...,...,...,...,...
51,KAF8242148.1,Wilcoxina mikolae CBS 423.85,418,MPALSPTMTAGNVGTWQKQPGDVLSPGDVLVEIETDKAQMDFEFQD...
198,KZF23348.1,Xylona heveae TC161,471,MSVATLLARGRRIILRNGPSLAALARCYASKSFPPHTVVSMPALSP...
181,XP_018188903.1,Xylona heveae TC161,471,MSVATLLARGRRIILRNGPSLAALARCYASKSFPPHTVVSMPALSP...
188,EGV65900.1,Yamadazyma tenuis ATCC 10573,462,MSLIGSTRTMALRIGSIPVTKRFMISSYQLARFYSSKFPNHVVINM...


In [34]:
if check_continue("\n...\n...\n...\nAll the data has been organized in a form, do you wanna generate a tsv file?"):
    df_all_data.to_csv("all_data_you_searched.tsv", sep = '\t', header = True)
    print("\ntsv file named 'all_data_you_searched.tsv' has been generated in the current working dictionary")
    


...
...
...
All the data has been organized in a form, do you wanna generate a tsv file? [yes/no]
	yes

tsv file named 'all_data_you_searched.tsv' has been generated in the current working dictionary


In [47]:
protein ="pyruvate_dehydrogenase"
taxonomy = "ascomycete_fungi"
winsize = 4

In [45]:
os.system(f"clustalo -i {protein}_{taxonomy}.fa -o {protein}_{taxonomy}_aligned.fa --distmat-out={protein}_{taxonomy}_distmat.mat --full")

0

In [48]:
os.system(f"plotcon {protein}_{taxonomy}_aligned.fa -winsize {winsize} -graph png")

Plot conservation of a sequence alignment


Created plotcon.1.png


0

In [53]:
image = Image.open("plotcon.1.png")
image.show()

Unable to init server: Could not connect: Connection refused

(eog:3906508): Gtk-WARNING **: 02:02:06.141: cannot open display: 


In [16]:
len(seqlen)


232

In [17]:
os.system("patmatmotifs")

Scan a protein sequence with motifs from the PROSITE database
Input protein sequence: Error: Unable to get reply from user - end of standard input


256